In [1]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
import numpy as np
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# 데이터 불러오기
click_stream = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/data_refine/click_stream_final.csv')
transaction = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/data_refine/transaction_final.csv')
product = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/data_refine/product_final.csv')
customer_add_rfm = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/customer_add_rfm.csv') # rfm seg 추가
sales = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/sales.csv')
df_session_info_rfm = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/df_session_info_rfm.csv')

In [3]:
df_buy_split = pd.read_csv('df_buy_split.csv') #customer id 있는 click_stream 세션 분할 파일
df_n_buy_split = pd.read_csv('df_n_buy_split.csv') #customer id 없는 click_stream 세션 분할 파일

In [4]:
customer_add_rfm.rename(columns = {'RFM_Segment_y': 'RFM_Segment'}, inplace = True)
customer_add_rfm.drop(columns= ['RFM_Segment_x'])

,customer_id,gender,birthdate,home_location,first_join_date,age_join,age_now,age_segment,join_year,join_month,RFM_Segment
0,2870,F,1996-06-14,Sumatera Barat,2019-07-21,23,26,Late 20s(26-30),2019,7,NaN
1,8193,F,1993-08-16,Jakarta Raya,2017-07-16,24,29,Late 20s(26-30),2017,7,NaN
2,7279,M,1989-01-23,Nusa Tenggara Barat,2020-08-23,31,33,Early 30s(31-35),2020,8,322.00
3,88813,M,1991-01-05,Kalimantan Timur,2021-10-03,30,31,Early 30s(31-35),2021,10,NaN
4,82542,M,2000-07-15,Kalimantan Selatan,2021-04-11,21,22,Early 20s(21-25),2021,4,111.00
...,...,...,...,...,...,...,...,...,...,...,...
99994,34743,M,2000-11-17,Lampung,2020-10-17,20,22,Early 20s(21-25),2020,10,322.00
99995,72065,M,1998-03-30,Aceh,2017-11-25,19,24,Early 20s(21-25),2017,11,322.00
99996,32818,F,1993-01-12,Maluku,2018-09-01,25,29,Late 20s(26-30),2018,9,322.00
99997,75597,M,2000-09-13,Jambi,2020-05-31,20,22,Early 20s(21-25),2020,5,NaN


# 1,2) 같은 물건 구매시 가격 high 10% low 10% 비율 특성

In [5]:
# 가격이 1개 이상인 데이터
price_variation_df = transaction.groupby("product_id")["item_price"].nunique()
multiple_price_product_id = price_variation_df[price_variation_df > 1].index
multiple_price_products = transaction[transaction["product_id"].isin(multiple_price_product_id)]
multiple_price_products


# 각 product_id별로 상위 10%, 하위 10%의 item_price를 계산
price_ranges = multiple_price_products.groupby('product_id')['item_price'].apply(lambda x: pd.Series({
    'low': x.quantile(0.1),
    'high': x.quantile(0.9)
})).reset_index()

# 각 product_id별로 상위 10%, 하위 10%의 item_price 추출
high_price = price_ranges[price_ranges['level_1']=='high'].rename(columns= {'item_price' : 'high'}).drop(columns='level_1')
low_price = price_ranges[price_ranges['level_1']=='low'].rename(columns= {'item_price' : 'low'}).drop(columns='level_1')

In [6]:
# 각 product_id별로 상위 10%, 하위 10%의 item_price 추가
range_merged1 = pd.merge(transaction, low_price, on = 'product_id', how = 'left')
range_merged2 = pd.merge(range_merged1, high_price, on = 'product_id', how = 'left')

# 각 product_id별로 상위 10%, 하위 10%의 item_price 비교하는 열 추가
range_merged2.loc[range_merged2.item_price <= range_merged2.low, 'price_category'] = 'low'
range_merged2.loc[range_merged2.item_price >= range_merged2.high, 'price_category'] = 'high'
range_merged2.loc[(range_merged2.item_price > range_merged2.low) & (range_merged2.item_price < range_merged2.high), 'price_category'] = 'normal'


In [7]:
rfm_final_tran = pd.merge(range_merged2, customer_add_rfm[['customer_id', 'RFM_Segment']], on = 'customer_id', how = 'left')
buy_range = rfm_final_tran.groupby('customer_id')['price_category'].value_counts()
buy_range = buy_range.unstack(level='price_category')


In [8]:
#데이터 타입 변경
rfm_final_tran['created_at'] = pd.to_datetime(rfm_final_tran['created_at'])

In [9]:
#고객별 상위 하위 10% 구매 횟수 카운트
total = pd.DataFrame()
for i in rfm_final_tran.customer_id.unique():
    for category in ['high', 'normal', 'low']:
        if category in buy_range.loc[i]:
            total.loc[i, category] = buy_range.loc[i][category]
        else:
            total.loc[i, category] = 0

#상위 하위 10% 가격 구매 비율 계산
ratio = pd.DataFrame()
for i in total.index:
    for category in ['high', 'normal', 'low']:
        if category in total.loc[i]:
            x = total.loc[i][category] / total.loc[i].sum()
            ratio.loc[i, category] = x


In [10]:
#결측값 처리
ratio.fillna(0,inplace= True)

In [11]:
ratio.reset_index(inplace= True)

In [12]:
ratio.rename(columns={'index' : 'customer_id'}, inplace= True)

In [13]:
#결과
ratio

,customer_id,high,normal,low
0,5868,0.00,1.00,0.00
1,4774,0.07,0.74,0.19
2,58191,0.13,0.75,0.12
3,81104,0.00,1.00,0.00
4,47040,0.00,0.67,0.33
...,...,...,...,...
50699,25121,0.00,1.00,0.00
50700,34829,0.00,1.00,0.00
50701,99645,0.00,1.00,0.00
50702,87319,0.00,1.00,0.00


# 3) 전체 구매횟수

In [15]:
#고객별 구매 횟수 계산하여 데이터 프레임으로 저장
total_book_cnt = pd.DataFrame(df_buy_split.groupby('customer_id').apply(lambda x: x[x.event_name == 'BOOKING'].session_id.nunique())).reset_index()
total_book_cnt.rename(columns = {0 : 'book_cnt'}, inplace =True)

In [17]:
#결과
total_book_cnt

,customer_id,book_cnt
0,3.00,51
1,8.00,7
2,9.00,6
3,11.00,1
4,15.00,5
...,...,...
50699,99992.00,12
50700,99995.00,60
50701,99997.00,1
50702,99998.00,8


# 4) 이벤트당 평균 체류 시간

In [18]:
#세션 별 이벤트당 체류시간 계산
df_session_info_rfm['time/event'] = df_session_info_rfm['session duration'] / df_session_info_rfm['number of events']

In [19]:
#고객 별로 평균 낸 데이터 프레임 생성
mean_event_time = pd.DataFrame(df_session_info_rfm[(df_session_info_rfm['number of events'] != 1)].groupby("customer_id").apply(lambda x: np.mean(x['time/event']))).reset_index()
mean_event_time.rename(columns = {0 : 'time/event'}, inplace =True)

In [20]:
#결과
mean_event_time

,customer_id,time/event
0,3.00,202.85
1,8.00,224.22
2,9.00,303.74
3,11.00,208.93
4,15.00,254.25
...,...,...
50699,99992.00,308.98
50700,99995.00,185.84
50701,99997.00,116.70
50702,99998.00,282.08


# 5) add_to_cart 에서 booking 으로 가는 시간

In [21]:
#데이터 타입 변경
df_buy_split.loc[:, 'event_time'] = pd.to_datetime(df_buy_split.loc[:, 'event_time'])

#함수 선언
def calculate_time_between_events(df):
    time_diffs = []
    prev_row = None
    for index, row in df.iterrows():
        if row.event_name == 'BOOKING' and prev_row is not None and prev_row.event_name == 'ADD_TO_CART':
            time_diff = row.event_time - prev_row.event_time
            time_diffs.append(time_diff.total_seconds())  # 초 단위로 시간 차이 저장
        prev_row = row
    return time_diffs

#함수 적용
add_book_diff = df_buy_split[(df_buy_split.event_name == 'ADD_TO_CART') | (df_buy_split.event_name == 'BOOKING')].groupby('customer_id').apply(lambda x: calculate_time_between_events(x.sort_values('event_time')))

#데이터 프레임으로 저장
add_book_diff_mean = pd.DataFrame()
for index in add_book_diff.index:
    add_book_diff_mean.loc[index, 'add_book_mean'] = np.mean(add_book_diff[index])/60/60/24
    add_book_diff_mean.loc[index, 'customer_id'] = index

add_book_diff_mean.reset_index(drop= True, inplace= True)

In [22]:
#결과
add_book_diff_mean

,add_book_mean,customer_id
0,3.56,3.00
1,8.58,8.00
2,10.62,9.00
3,0.01,11.00
4,32.27,15.00
...,...,...
50699,7.12,99992.00
50700,2.69,99995.00
50701,0.00,99997.00
50702,12.00,99998.00


# 6) 재방문 주기

In [23]:
#함수 선언
def revisit(df):
    df.loc[:,"event_time"] = pd.to_datetime(df.loc[:,"event_time"])
    df = df.sort_values(by=['customer_id', 'event_time'])
    # customer_id별 한 session의 첫번째 이벤트
    df_head = df.groupby(['customer_id', 'session_id'])[['customer_id', 'session_id', 'event_time']].head(1)
    # customer_id별 한 session의 마지막 이벤트
    df_tail = df.groupby(['customer_id', 'session_id'])[['customer_id', 'session_id', 'event_time']].tail(1)

    # customer_id별 한 session의 첫번째, 마지막 이벤트 나열
    df_H_T = pd.concat([df_head, df_tail], axis = 0)
    df_H_T = df_H_T.sort_values(['customer_id', 'event_time'])
    
    # 이벤트 간 차이 구하기
    df_H_T['revisit'] = df_H_T.groupby('customer_id')['event_time'].diff()

    # 이전 session의 마지막 이벤트와 현재 session의 첫번째 이벤트 간 시간 차이(짝수번째 행들에 나열되어 있음)
    df_odd_rows = df_H_T.groupby('customer_id').apply(lambda x: x.iloc[::2]).reset_index(drop=True)
    
    return df_odd_rows.groupby('customer_id').revisit.mean()

In [24]:
#함수 적용
revist_res =  revisit(df_buy_split[['session_id', 'customer_id', 'event_time']])

In [25]:
#결측값 처리
revist_res_df = pd.DataFrame(revist_res).reset_index().fillna(0)

In [26]:
#결과
revist_res_df

,customer_id,revisit
0,3.00,2 days 11:22:20.800353356
1,8.00,2 days 10:35:21.481283422
2,9.00,8 days 09:08:41.075000
3,11.00,0
4,15.00,21 days 05:33:16.974358974
...,...,...
50699,99992.00,11 days 04:24:32.806451612
50700,99995.00,0 days 23:11:52.696306429
50701,99997.00,0
50702,99998.00,3 days 23:16:17.314024390


# 7) 세션 수(방문수)

In [27]:
#세션수 계산하여 데이터 프레임으로 저장
df_session_g = pd.DataFrame(df_session_info_rfm[df_session_info_rfm['session_id'].isin(df_buy_split.session_id)].groupby('customer_id')['session_id'].nunique()).reset_index()
df_session_g.rename(columns = {'session_id' : 'session_cnt'}, inplace =True)

In [28]:
#결과
df_session_g

,customer_id,session_cnt
0,3.00,567
1,8.00,188
2,9.00,41
3,11.00,1
4,15.00,40
...,...,...
50699,99992.00,63
50700,99995.00,732
50701,99997.00,1
50702,99998.00,329


# 8) 세션 이벤트 마지막이 add to cart 인 비율

In [29]:
#데이터 정렬하기
df_buy_split.sort_values(['customer_id', 'event_time'], inplace = True)

# 각 세션의 마지막 이벤트 추출
last_events = df_buy_split.groupby(['customer_id', 'session_id'])['event_name'].last().reset_index()

# 이벤트별 비율 계산하여 데이터 프레임으로 저장
event_counts = pd.DataFrame(last_events.groupby('customer_id')['event_name'].value_counts())
event_counts.reset_index(inplace= True)

In [30]:
#결과
event_counts

,customer_id,event_name,count
0,3.00,ADD_TO_CART,113
1,3.00,HOMEPAGE,102
2,3.00,CLICK,98
3,3.00,SCROLL,60
4,3.00,BOOKING,51
...,...,...,...
336612,99998.00,PROMO_PAGE,16
336613,99998.00,SEARCH,15
336614,99998.00,BOOKING,8
336615,99998.00,ADD_PROMO,4


In [31]:
# "ADD_TO_CART" 이벤트만 추출
add_events = event_counts[event_counts['event_name'] == 'ADD_TO_CART'][['customer_id', 'count']].rename(columns={'count': 'add_cnt'})

# customer_id 별로 그룹화하여 총 이벤트 수와 "BOOKING" 이벤트의 총 합 계산
grouped = event_counts.groupby('customer_id')['count'].sum().reset_index().rename(columns={'count': 'total_cnt'})

# 각 그룹에서 "ADD_TO_CART" 이벤트의 총 합을 전체 이벤트 수로 나누어 비율 계산
merge_df = pd.merge(grouped, add_events, on='customer_id', how='left')
merge_df['ratio'] = merge_df['add_cnt'] / merge_df['total_cnt']
merge_df.rename(columns={'ratio' : 'add_last_event_ratio'}, inplace= True)

In [32]:
#결측값 처리
merge_df.fillna(0, inplace= True)

In [33]:
#결과
merge_df

,customer_id,total_cnt,add_cnt,add_last_event_ratio
0,3.00,567,113.00,0.20
1,8.00,188,10.00,0.05
2,9.00,41,9.00,0.22
3,11.00,1,0.00,0.00
4,15.00,40,12.00,0.30
...,...,...,...,...
50699,99992.00,63,13.00,0.21
50700,99995.00,732,37.00,0.05
50701,99997.00,1,0.00,0.00
50702,99998.00,329,35.00,0.11


# 9) promotion 이용 비율

In [53]:
#함수 선언
def promo_ratio_create(df):
    #booking id 별로 반복
    df_g = df.groupby('booking_id')
    promo_code_count = 0
    
    #promo code 가 0이 아닌 경우 프로모션 사용이므로 이때 개수를 측정
    for b_i, row in df_g:
        if row['promo_code'].any() != 0:
            promo_code_count += 1
    
    res = promo_code_count/df.booking_id.nunique()
    return res

#함수 적용
promo_ratio = df_buy_split.groupby('customer_id').apply(lambda x: promo_ratio_create(x))


In [54]:
#결과 데이터 프레임으로 저장
promo_ratio_df = pd.DataFrame(promo_ratio).reset_index()
promo_ratio_df.rename(columns= {0 : 'promo_use'}, inplace= True)

In [55]:
#결과
promo_ratio_df

,customer_id,promo_use
0,3.00,0.53
1,8.00,0.14
2,9.00,0.00
3,11.00,1.00
4,15.00,0.80
...,...,...
50699,99992.00,0.17
50700,99995.00,0.22
50701,99997.00,0.00
50702,99998.00,0.75


# 10) 객단가

In [46]:
#Booking 데이터와 매출 데이터 병합
book_df = df_buy_split[df_buy_split.event_name == 'BOOKING']
book_sales_df = pd.merge(book_df, sales[['booking_id', 'sales']].drop_duplicates(), on = 'booking_id', how = 'left')

In [47]:
#고객별로 총매출 계산
customer_sales_ratio = book_sales_df.groupby('customer_id').apply(lambda x: x['sales'].sum()/x['booking_id'].nunique())

In [48]:
#결과 데이터 프레임으로 저장
customer_sales_ratio_df = pd.DataFrame(customer_sales_ratio).reset_index()
customer_sales_ratio_df.rename(columns= {0 :'average_revenue'}, inplace= True)

In [49]:
#결과
customer_sales_ratio_df

,customer_id,average_revenue
0,3.00,409821.35
1,8.00,550508.71
2,9.00,433944.17
3,11.00,182533.00
4,15.00,416974.00
...,...,...
50699,99992.00,351852.42
50700,99995.00,591562.38
50701,99997.00,216461.00
50702,99998.00,829692.50


# 최종 데이터

In [56]:
#이탈 데이터 불러오기
bounce_25_book = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/bounce_25_book.csv')
bounce_25_book.rename(columns={'Unnamed: 0' : 'customer_id'}, inplace= True)

In [57]:
# 학습 데이터 프레임 생성
feature_target = customer_add_rfm[['customer_id', 'gender', 'home_location', 'first_join_date', 'age_segment', 'RFM_Segment']].copy()
feature_target.dropna(inplace = True)


In [58]:
# 각 특성 추가
feature_target = pd.merge(feature_target, ratio[['customer_id', 'low', 'high']], on = 'customer_id', how = 'left').fillna(0)
feature_target = pd.merge(feature_target, total_book_cnt, on = 'customer_id', how = 'left').fillna(0)
feature_target = pd.merge(feature_target, mean_event_time, on = 'customer_id', how = 'left').fillna(0)
feature_target = pd.merge(feature_target, add_book_diff_mean, on = 'customer_id', how = 'left').fillna(0)
feature_target = pd.merge(feature_target, revist_res_df, on = 'customer_id', how = 'left').fillna(0)
feature_target = pd.merge(feature_target, df_session_g, on = 'customer_id', how = 'left').fillna(0)
feature_target = pd.merge(feature_target, merge_df[['customer_id', 'add_cnt']], on = 'customer_id', how = 'left').fillna(0)
feature_target = pd.merge(feature_target, promo_ratio_df, on = 'customer_id', how = 'left').fillna(0)
feature_target = pd.merge(feature_target, customer_sales_ratio_df, on = 'customer_id', how = 'left').fillna(0)

##

In [59]:
#마지막 달의 이탈 추가(예측 타겟)
feature_target['bounce'] = bounce_25_book['2022-07']
feature_target

,customer_id,gender,home_location,first_join_date,age_segment,RFM_Segment,low,high,book_cnt,time/event,add_book_mean,revisit,session_cnt,add_cnt,promo_use,average_revenue,bounce
0,7279,M,Nusa Tenggara Barat,2020-08-23,Early 30s(31-35),322.00,0.14,0.07,89,104.58,1.67,0 days 17:06:10.890688259,989,59.00,0.20,537850.17,1
1,82542,M,Kalimantan Selatan,2021-04-11,Early 20s(21-25),111.00,1.00,0.00,1,108.30,0.00,0,1,0.00,0.00,115702.00,1
2,96453,F,Bengkulu,2022-01-09,minor(15-20),211.00,0.00,0.00,2,260.64,2.80,10 days 05:37:20.600000,11,4.00,0.50,190362.00,1
3,8031,F,Jawa Barat,2019-09-12,Late 20s(26-30),111.00,0.50,0.25,3,311.72,5.75,43 days 21:52:05.125000,17,4.00,1.00,770944.33,1
4,9152,F,Yogyakarta,2019-11-21,Late 20s(26-30),111.00,0.33,0.00,3,261.24,24.36,46 days 21:04:21.714285714,15,5.00,0.67,163779.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50694,38690,F,Riau,2017-07-30,non-target adults(36-),111.00,0.14,0.14,6,287.88,41.05,41 days 02:11:44.972972973,38,8.00,0.50,568302.33,1
50695,34743,M,Lampung,2020-10-17,Early 20s(21-25),322.00,0.13,0.20,37,141.35,3.71,1 days 02:40:20.154373927,584,20.00,0.32,429829.43,1
50696,72065,M,Aceh,2017-11-25,Early 20s(21-25),322.00,0.08,0.08,44,223.70,5.94,4 days 09:55:35.168421052,381,86.00,0.30,702686.39,1
50697,32818,F,Maluku,2018-09-01,Late 20s(26-30),322.00,0.09,0.13,96,124.38,2.87,1 days 02:52:14.758647798,1273,78.00,0.38,590131.12,1


특성 정리<br/>
customer_id : 고객아이디<br/>
gender : 성별<br/>
hom_location : 지역위치<br/>
first_join_date : 처음 가입 날짜<br/>
age_segment : 나이 그룹<br/>
RFM_Segment : RFM 그룹<br/>
low : 하위 10% 로 구매한 비율 <br/>
high : 상위 10% 로 구매한 비율<br/>
book_cnt : 구매 횟수<br/>
time / event : 이벤트 평균 길이(s)<br/>
add_book_mean : add_to_cart 에서 booking 사이에 평균 시간 (days)<br/>
revisit : 평균 재방문 주기<br/>
session_cnt : 방문횟수(session 수)<br/>
add_last_event_ratio : add_to_cart 로 끝나는 세션의 비율<br/>
promo_use_ratio : 프로모션 적용한 구매 비율<br/>
average_revenue : 세션당 객단가 <br/>
bounce : 이탈 여부 (target)<br/>

In [60]:
#학습 데이터 내보내기
feature_target.to_csv('feature_target.csv', index = False)